In [1]:
import numpy as np
import MDAnalysis as md
from tqdm import tqdm_notebook as tqdm

Compute (second rank) order parameter, defined as:

\begin{equation*}
P2 = 0.5*(3*<cos²(\theta)> - 1)
\end{equation*}

where "theta" is the angle between the bond and the bilayer normal.
P2 = 1      perfect alignement with the bilayer normal
P2 = -0.5   anti-alignement
P2 = 0      random orientation
By default P2 will be calculated relative to the z-axis (the vector 0 0 1)
This can be changed, by simply changing the vector reffered to as normal

There are two functions to calculate the ordere parameter.
One for calcualting it for all the lipids in the universe, and another for calculating it for a selection of lipids.
For the selection of lipids; They are split up in two group based on a cutoff value in angstrom from the scaffold proteins. This is for the Nanodiscs. 

In [2]:
name = '1D1_60_v1_nowat'


u = md.Universe('TPR/{0:s}.tpr'.format(name), 'XTC/{0:s}_fit.xtc'.format(name))
lipid_type =  'POPC' # the lipid type
cutoff = 15 # cutoff for selection of lipids - center vs edge
normal = np.array([0, 0, 1]) #normal vector
dt = 1 #stride


In [3]:
number_of_lipids =  np.unique(u.select_atoms('resname {0:s}'.format(lipid_type)).resids).shape[0]
orientation_of_bilayer_normal = normal
norm = np.linalg.norm(orientation_of_bilayer_normal)
orientation_of_bilayer_normal / norm

array([0., 0., 1.])

In [4]:
phosphatidylcholine_bond_names = " NC3-PO4 PO4-GL1 GL1-GL2 "
phosphatidylethanolamine_bond_names = " NH3-PO4 PO4-GL1 GL1-GL2 "
# PCs
if   lipid_type == "DAPC": bond_names = phosphatidylcholine_bond_names + "GL1-D1A GL2-D1B D1A-D2A D2A-D3A D3A-D4A D4A-C5A D1B-D2B D2B-D3B D3B-D4B D4B-C5B\n"
elif lipid_type == "DLPC": bond_names = phosphatidylcholine_bond_names + "GL1-C1A GL2-C1B C1A-C2A C2A-C3A C1B-C2B C2B-C3B\n"
elif lipid_type == "DOPC": bond_names = phosphatidylcholine_bond_names + "GL1-C1A GL2-C1B C1A-D2A D2A-C3A C3A-C4A C1B-D2B D2B-C3B C3B-C4B\n"
elif lipid_type == "DPPC": bond_names = phosphatidylcholine_bond_names + "GL1-C1A GL2-C1B C1A-C2A C2A-C3A C3A-C4A C1B-C2B C2B-C3B C3B-C4B\n"
elif lipid_type == "POPC": bond_names = phosphatidylcholine_bond_names + "GL1-C1A GL2-C1B C1A-D2A D2A-C3A C3A-C4A C1B-C2B C2B-C3B C3B-C4B\n"
elif lipid_type == "POPC_Martini3" : bond_names = phosphatidylcholine_bond_names + "GL2-C1B C1B-C2B C2B-C3B C3B-C4B GL1-C1A C1A-D2A D2A-C3A C3A-C4A\n"
# PEs
elif lipid_type == "DLPE": bond_names = phosphatidylethanolamine_bond_names + "GL1-C1A GL2-C1B C1A-C2A C2A-C3A C1B-C2B C2B-C3B\n"
elif lipid_type == "DOPE": bond_names = phosphatidylethanolamine_bond_names + "GL1-C1A GL2-C1B C1A-D2A D2A-C3A C3A-C4A C1B-D2B D2B-C3B C3B-C4B\n"
elif lipid_type == "DPPE": bond_names = phosphatidylethanolamine_bond_names + "GL1-C1A GL2-C1B C1A-C2A C2A-C3A C3A-C4A C1B-C2B C2B-C3B C3B-C4B\n"
elif lipid_type == "POPE": bond_names = phosphatidylethanolamine_bond_names + "GL1-C1B GL2-C1A C1A-C2A C2A-C3A C3A-C4A C1B-D2B D2B-C3B C3B-C4B\n"


In [5]:
def calc_order_ver1 (u, lipid_type, bond_names, orientation_of_bilayer_normal):
    """Outputs an array with an order parameter for each bond in the lipid averaged
    over the number of lipids - does it for all the lipids in the universe"""
    
    order_parameters = []
    bonds = []
    current_order_parameters = []
    
    number_of_lipids =  np.unique(u.select_atoms('resname {0:s}'.format(lipid_type)).resids).shape[0]
    
    for bond_name in bond_names.split():
        bonds.append(bond_name.split("-"))
    
    for bond in bonds:
        print bond
        first = u.select_atoms('resname {0:s} and name {1:s}'.format(lipid_type, bond[0])).positions
        second = u.select_atoms('resname {0:s} and name {1:s}'.format(lipid_type, bond[1])).positions
        order_parameter = 0.0
        for i in range(number_of_lipids):
            vector = first[i] - second[i]
            norm2 = vector[0]**2 + vector[1]**2 + vector[2]**2
            projection = vector[0]*orientation_of_bilayer_normal[0] + vector[1]*orientation_of_bilayer_normal[1] + vector[2]*orientation_of_bilayer_normal[2]
            order_parameter += projection**2/norm2
        current_order_parameters.append(0.5*(3.0*(order_parameter/number_of_lipids) - 1.0))
    order_parameters.append(current_order_parameters)

    return np.array(order_parameters)

In [6]:
def calc_order_ver2 (sel, bond_names, orientation_of_bilayer_normal):
    """Outputs an array with an order parameter for each bond in the lipid averaged
    over the number of lipids - does it for a selection of lipids in the universe"""
    
    order_parameters = []
    bonds = []
    current_order_parameters = []
    
    number_of_lipids =  np.unique(sel.resids).shape[0]
    
    for bond_name in bond_names.split():
        bonds.append(bond_name.split("-"))
    
    for bond in bonds:
        first = sel.select_atoms('name {0:s}'.format(bond[0])).positions
        second = sel.select_atoms('name {0:s}'.format(bond[1])).positions
        order_parameter = 0.0
        for i in range(number_of_lipids):
            vector = first[i] - second[i]
            norm = vector[0]**2 + vector[1]**2 + vector[2]**2
            projection = vector[0]*orientation_of_bilayer_normal[0] + vector[1]*orientation_of_bilayer_normal[1] + vector[2]*orientation_of_bilayer_normal[2]
            order_parameter += projection**2/norm
        current_order_parameters.append(0.5*(3.0*(order_parameter/number_of_lipids) - 1.0))
    order_parameters.append(current_order_parameters)

    return np.array(order_parameters)

In [7]:
def get_whole_lipids_sel (sel, u, cutoff):
    """A function to get an atomselect, such that the lipids are whole"""
    #print sel
    sele = u.select_atoms(sel)
    r1 = np.unique(sele.resids)
    if r1.shape[0] == 0:
        print 'Cutoff value to big, all lipids are within {0:d} of protein'.format(cutoff)
    r2 = [ "{0:d}".format(i) for i in r1 ]
    resids = " ".join(r2)
    whole_sel = u.select_atoms('resid {0:s}'.format(resids))
    return whole_sel

In [8]:
bonds = []

for bond_name in bond_names.split():
    bonds.append(bond_name.split("-"))


In [9]:
nframes = u.trajectory.n_frames
nframes = nframes / dt 

In [10]:
n_order = len(bonds)

In [11]:
order_traj_edge = np.zeros([nframes, n_order])
order_traj_cent = np.zeros([nframes, n_order])

In [12]:
s = u.select_atoms('resname POPC and not (resname POPC and around 30 protein)')
np.unique(s.resids).shape[0]

23

In [13]:
u.select_atoms('resname POPC and name PO4 and around 30 protein')

<AtomGroup with 107 atoms>

In [14]:
    for ts in u.trajectory[1::dt]:
        number_of_lipids_edge = np.unique(u.select_atoms('resname {0:s} and around {1:d} protein'.format(lipid_type, cutoff)).resids).shape[0]
        number_of_lipids_cent = np.unique(u.select_atoms('resname {0:s} and not (resname {0:s} and around {1:d} protein)'.format(lipid_type, cutoff)).resids).shape[0]

        frame = u.trajectory.frame/ dt
        #print frame

        edge = 'resname {0:s} and global around {1:d} protein'.format(lipid_type, cutoff)
        #print edge
        cent = 'resname {0:s} and not (resname {0:s} and around {1:d} protein)'.format(lipid_type, cutoff)
        #print cent

        edge_sel = get_whole_lipids_sel(edge, u, cutoff)
        cent_sel = get_whole_lipids_sel(cent, u, cutoff)

        order_traj_edge[frame] = calc_order_ver2(edge_sel, bond_names, orientation_of_bilayer_normal)
        order_traj_cent[frame] = calc_order_ver2(cent_sel, bond_names, orientation_of_bilayer_normal)

In [15]:
np.mean(order_traj_cent)

0.18648776194458105

In [16]:
np.mean(order_traj_edge)

0.11785539555736892

In [17]:
file_out  = open('ORDER/{0:s}_Average_OrderP2_edge_cent.dat'.format(name), 'w')
file_out.write('{0:s}, used cutoff of {1:d} Angstrom\n'.format(name, cutoff))
file_out.write('{0:s}\n'.format(bond_names))
file_out.write('Average order parameter for Edge\n')
file_out.write('{0:s}\n'.format(" ".join(["{0:3.2f}".format(i) for i in np.mean(order_traj_edge, axis=0)])))
file_out.write('Average order parameter for Cent\n')
file_out.write('{0:s}\n'.format(" ".join(["{0:3.2f}".format(i) for i in np.mean(order_traj_cent, axis=0)])))


In [18]:
file_out.close()